<a href="https://colab.research.google.com/github/Shiyao-monster/web/blob/main/sentence_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 1024 # max length of the review
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17473536/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the nda

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 128),
      tf.keras.layers.LSTM(32, return_sequences=True),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])
else:
  print('Found GPU at: {}'.format(device_name))
  with tf.device('/device:GPU:0'):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 128),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
        ])


In [3]:
# model = tf.keras.Sequential([
#       tf.keras.layers.Embedding(VOCAB_SIZE, 128),
#       tf.keras.layers.LSTM(32, return_sequences=True),
#       tf.keras.layers.LSTM(32),
#       tf.keras.layers.Dense(1, activation='sigmoid')
#       ])

In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)


# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc'])

model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=['acc'])

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               monitor='val_acc',
                                                               mode='max',
                                                               save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)

history = model.fit(train_data, train_labels, epochs=8, validation_split=0.2, 
                    callbacks=[model_checkpoint_callback, early_stopping_callback])

!mkdir -p drive
!google-drive-ocamlfuse drive
import sys
sys.path.append('drive/Project')
model.save('drive/Project/latest_model.h5')

In [8]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 116s 149ms/step - loss: 0.3028 - acc: 0.8816
[0.3028259575366974, 0.881600022315979]


In [ ]:
from keras.models import load_model

model.load_weights(checkpoint_filepath)

!mkdir -p drive
!google-drive-ocamlfuse drive
sys.path.append('drive/Project')
model.save('drive/Project/best_model.h5')

word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,1024))
  pred[0] = encoded_text
  result = model.predict(pred) 
  # print(result[0])
  return result


In [ ]:
# positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
positive_review = "I am grateful."
predict(positive_review).item()

In [10]:
best_model = load_model('/content/drive/Project/best_model.h5')

In [11]:
best_results = best_model.evaluate(test_data, test_labels)
print(best_results)

782/782 [==============================] - 129s 146ms/step - loss: 0.3028 - acc: 0.8816
[0.3028259575366974, 0.881600022315979]


In [12]:
latest_model = load_model('/content/drive/Project/latest_model.h5')
latest_results = latest_model.evaluate(test_data, test_labels)
print(latest_results)

782/782 [==============================] - 119s 151ms/step - loss: 0.4982 - acc: 0.8681
[0.49821633100509644, 0.868120014667511]


In [14]:
def best_predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,1024))
  pred[0] = encoded_text
  result = best_model.predict(pred) 
  print(result[0])
  return result
positive_review = "I am happy."
best_predict(positive_review).item()

[0.07327387]


0.07327386736869812

In [15]:
def latest_predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,1024))
  pred[0] = encoded_text
  result = latest_model.predict(pred) 
  print(result[0])
  return result
positive_review = "I am happy."
latest_predict(positive_review).item()

[0.02598873]


0.025988727807998657